# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
from collections import deque
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.0


When finished, you can close the environment.

In [9]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [10]:
from ddpg_agent import Agent, ReplayBuffer, GaussianNoise
from ddpg_model import Actor, Critic
from ddpg_trainer import DDPGTrainer
from torch import optim
import torch
import time
import numpy as np

In [11]:
SEED = 42
device = 'cuda:0' if torch.cuda.is_available() else 'cpu:0'

In [12]:
# Extract state dimensionality from env.
brain_name = env.brain_names[0]
env_info = env.reset(train_mode=True)[brain_name]
state_size = env_info.vector_observations.shape[-1]

# Extract action dimensionality and number of agents from env.
brain = env.brains[brain_name]
action_size = brain.vector_action_space_size
num_agents = len(env_info.agents)

# Display relevant environment information.
print(f'Number of agents: {num_agents},'
      + f'state size: {state_size} '
      + f', action size: {action_size}')

Number of agents: 2,state size: 24 , action size: 2


In [13]:
batch_size = 1024
# Initialize the replay buffer from which experiences are gathered for
# training the agent.
replay_buffer = ReplayBuffer(
    seed=SEED,
    buffer_size=int(1e5),
    batch_size=batch_size
)

In [14]:
# Initialize local and target Actor Networks and optimizer.
actor_local = Actor(state_size, action_size, SEED,
                    400, 300).to(device)
actor_target = Actor(state_size, action_size, SEED,
                        400, 300).to(device)
actor_optimizer = optim.Adam(actor_local.parameters(), lr=.000025)

# Initialize local and target Critic Networks and optimizer.
critic_local = Critic(state_size, action_size, SEED,
                        400, 300).to(device)
critic_target = Critic(state_size, action_size, SEED,
                        400, 300).to(device)
critic_optimizer = optim.Adam(critic_local.parameters(), lr=0.00025,
                                weight_decay=0)

# Initialize local and target Actor Networks and optimizer.
actor_local_2 = Actor(state_size, action_size, SEED,
                    400, 300).to(device)
actor_target_2 = Actor(state_size, action_size, SEED,
                        400, 300).to(device)
actor_optimizer_2 = optim.Adam(actor_local.parameters(), lr=.000025)

# Initialize local and target Critic Networks and optimizer.
critic_local_2 = Critic(state_size, action_size, SEED,
                        400, 300).to(device)
critic_target_2 = Critic(state_size, action_size, SEED,
                        400, 300).to(device)
critic_optimizer_2 = optim.Adam(critic_local.parameters(), lr=0.00025,
                                weight_decay=0)

# Initialize Gaussian noise to reduce generalization error.
noise = GaussianNoise(action_size, SEED, mu=0.0, sigma=0.3)

# Create agent object used for training.
agent_0 = Agent(
    seed=SEED,
    memory=replay_buffer,
    batch_size=batch_size,
    actor_local=actor_local,
    actor_target=actor_target,
    actor_optimizer=actor_optimizer,
    critic_local=critic_local,
    critic_target=critic_target,
    critic_optimizer=critic_optimizer,
    noise=noise,
    gamma=1,
    tau=0.001
)

agent_1 = Agent(
    seed=SEED,
    memory=replay_buffer,
    batch_size=batch_size,
    actor_local=actor_local_2,
    actor_target=actor_target_2,
    actor_optimizer=actor_optimizer_2,
    critic_local=critic_local_2,
    critic_target=critic_target_2,
    critic_optimizer=critic_optimizer_2,
    noise=noise,
    gamma=1,
    tau=0.001
)

# Initialize DDPGTrainer object with relevant arguments.
trainer = DDPGTrainer(
    env=env,
    update_frequency=1,
    num_updates=5,
    max_episode_length=1000,
    save_dir='save_dir',
    score_window_size=100)

trainer.agents.append(agent_0)
trainer.agents.append(agent_1)

In [15]:
actions

array([[-0.12229451, -0.62025894],
       [-0.40697375,  0.15452493]])

In [16]:
n_episodes = 10000
score_window_size = 100
target_score = 1
# Initialize timer and notify of training start.
t_start = time.time()
print('Starting training...')

# Train the agent for n_episodes.
for i in range(1, n_episodes + 1):
    for agent_index in range(1, 3):
        # Step through the training process and notify progress
        trainer.train_step()
        trainer.print_status(put_new_line=False)

        # After every 10 episodes, initiate new line of notification.
        if trainer.i_episode % 10 == 0:
            trainer.print_status(put_new_line=True)

        # If mean score over last score_window_size episodes exceeds
        # target score, plot training progress and save trainer
        if len(trainer.scores) > score_window_size and \
                trainer.get_running_mean_score() > target_score:
            trainer.plt_mavg()
            print('\nEnvironment is solved.')
            print('Saving trainer...')
            trainer.save()
            print('Done.')
            break

# Close environment and notify of training time.
print('\nFinished training, closing env')
env.close()
t_end = time.time()
delta = t_end - t_start
minutes = delta / 60
print(f'Training took {minutes:.1f} minutes')

Starting training...
Episode 10	Average Score: 0.04
Episode 20	Average Score: 0.02
Episode 30	Average Score: 0.03
Episode 40	Average Score: 0.02
Episode 50	Average Score: 0.02
Episode 60	Average Score: 0.02
Episode 70	Average Score: 0.02
Episode 80	Average Score: 0.02
Episode 90	Average Score: 0.02
Episode 100	Average Score: 0.02
Episode 110	Average Score: 0.02
Episode 120	Average Score: 0.03
Episode 130	Average Score: 0.03
Episode 140	Average Score: 0.03
Episode 150	Average Score: 0.03
Episode 160	Average Score: 0.03
Episode 170	Average Score: 0.03
Episode 180	Average Score: 0.02
Episode 190	Average Score: 0.03
Episode 200	Average Score: 0.02
Episode 210	Average Score: 0.02
Episode 220	Average Score: 0.02
Episode 230	Average Score: 0.02
Episode 240	Average Score: 0.02
Episode 250	Average Score: 0.02
Episode 260	Average Score: 0.02
Episode 270	Average Score: 0.02
Episode 278	Average Score: 0.02

In [ ]:
rewards

In [ ]:
states.shape[0]

In [ ]:
x = 0

In [ ]:
x = (x + 1) % 2
print(x)

In [ ]:
x += 1
print(x)

In [ ]:
states.shape